In [2]:
import pandas as pd
from surprise import SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

In [3]:
names = pd.read_csv('names.csv')

In [77]:
names.head(10)

,7 Days to Die,A Hat in Time,A Plague Tale: Innocence,A Way Out,ARK,ASMR,ATLAS,Agar.io,Age of Empires II,Age of Mythology,...,Wrestling,XERA: Survival,Yakuza 0,Yakuza Kiwami 2,Yu-Gi-Oh! Duel Links,Yu-Gi-Oh! Legacy of the Duelist,Yu-Gi-Oh! TRADING CARD GAME,iRacing.com,osu!,theHunter: Call of the Wild
user_name,,,,,,,,,,,,,,,,,,,,,
001pickle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007_zay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
008kool2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00NothingLabs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00elu00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00fero00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ironskull,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00lofi21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00nanni00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
names.set_index('user_name',inplace=True)

In [5]:
names = names.astype('float16')

In [6]:
names.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122220 entries, 001pickle to ｵｯｻﾝ御前
Columns: 331 entries, 7 Days to Die to theHunter: Call of the Wild
dtypes: float16(331)
memory usage: 78.1+ MB


In [7]:
import numpy as np
from scipy import sparse

In [8]:
matrix_sparse = sparse.csr_matrix(names)

In [9]:
coo = matrix_sparse.tocoo(copy=False)

In [86]:
df = pd.DataFrame({'user': coo.row, 'game': coo.col, 'views_over_max_views': coo.data}
                 )[['user', 'game', 'views_over_max_views']].sort_values(['user', 'game']
                 ).reset_index(drop=True)

In [32]:
from sklearn.preprocessing import minmax_scale

In [84]:
df.head(10)

,user,game,data
0,0,205,4.0
1,1,170,2.0
2,2,273,4.0
3,3,170,2.0
4,3,292,24.0
5,4,68,12.0
6,5,313,4.0
7,6,67,3.0
8,7,264,4.0
9,8,171,8.0


In [60]:
names.index[0]

'001pickle'

In [56]:
names.columns[205]

'ROBLOX'

In [62]:
df.iloc[0]['index']

0.0

In [63]:
df.iloc[0]['col']

205.0

In [67]:
df['user'] = ''
df['game'] = ''

In [69]:
for i, row in df.iterrows():
    df.iloc[i]['user'] = names.index[i]
    df.iloc[i]['game'] = names.columns[df.iloc[i]['col']]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [34]:
df['data_scaled'] = minmax_scale(df['data'], feature_range=(1, 5))

In [35]:
df.head(3)

,index,col,data,data_scaled
0,0,205,4.0,1.027088
1,1,170,2.0,1.009029
2,2,273,4.0,1.027088


In [36]:
lower_bound = df['data_scaled'].min()
higher_bound = df['data_scaled'].max()
print(lower_bound, higher_bound)

1.0 5.0


In [37]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import surprise
from surprise import SVDpp

import random

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [39]:
reader = surprise.Reader(rating_scale = (1, 5))

In [40]:
data = Dataset.load_from_df(df[['index', 'col', 'data_scaled']], reader)

In [46]:
alg = SVDpp(verbose=True)

In [47]:
cross_validate(alg, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1584  0.1585  0.1551  0.1563  0.1563  0.1569  0.0013  
MAE (testset)     0.0834  0.0836  0.0833  0.0832  0.0830  0.0833  0.0002  
Fit time          19.67   21.94   22.44   21.07   15.89   20.20   2.35    
Test time         0.43    0.43    0.47    0.31    0.22    0.37    0.09    


{'test_rmse': array([0.15840308, 0.1584925 , 0.15506537, 0.15625394, 0.15634922]),
 'test_mae': array([0.08339212, 0.08363577, 0.08326193, 0.08320479, 0.0830235 ]),
 'fit_time': (19.66711688041687,
  21.9421968460083,
  22.438787937164307,
  21.06731605529785,
  15.894592046737671),
 'test_time': (0.43090200424194336,
  0.43132519721984863,
  0.47439098358154297,
  0.3079187870025635,
  0.22489595413208008)}

In [48]:
trainset, testset = train_test_split(data, test_size=.25)

# Train the algorithm on the trainset, and predict ratings for the testset
alg.fit(trainset)
predictions = alg.test(testset)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19


In [49]:
accuracy.rmse(predictions)

RMSE: 0.1597


0.1597096927323896

In [75]:
testset[1]

(105260, 231, 1.1173815727233887)

In [76]:
uid = 54239
iid = 231


pred = alg.predict(uid, iid, verbose=True)

user: 54239      item: 231        r_ui = None   est = 1.04   {'was_impossible': False}
